# LAB-GPU EJERCICIO 2

### Librerias

In [1]:
import numpy as np
import cupy as cp
import time
from numba import float32, cuda
from IPython import get_ipython

In [10]:
# Funcion usando el método secuencial

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Definir la operación de reducción con cuda

@cuda.jit
def gpu_reduc_operation(arr, result):
    """
    Suma los elementos de un array utilizando CUDA sin paralelizar el proceso.
    Se ejecuta en un único hilo.
    """
    if cuda.threadIdx.x == 0 and cuda.blockIdx.x == 0:  # Asegurar un único hilo
        temp_sum = 0.0
        for i in range(arr.size):  # Suma secuencial en la GPU
            temp_sum += arr[i]
        result[0] = temp_sum

def reduc_operation_gpu(arr):
    """
    Realiza la suma total del array utilizando Numba CUDA y finaliza con np.sum.
    """
    if not isinstance(arr, np.ndarray):
        raise ValueError("La entrada debe ser un array de NumPy")
    
    # Convertir a float32 para asegurar compatibilidad con CUDA
    arr = arr.astype(np.float32)

    # Transferir el array a la GPU
    d_arr = cuda.to_device(arr)
    d_result = cuda.to_device(np.zeros(1, dtype=np.float32))

    # Copiar el resultado de vuelta a la CPU
    partial_sum = d_result.copy_to_host()

    # Usar np.sum para completar la operación
    return np.sum(partial_sum)

### Variables

In [3]:
# Definir la variable value

value = None

# Intentar obtener el valor de la variable VALUE pasada como un argumento

try:
    # Obtener el valor de la variable VALUE definida en el entorno
    ipython = get_ipython()
    value = int(ipython.getoutput("echo $VALUE")[0].strip())
    if value <= 0:
        raise ValueError("El valor de VALUE debe ser un entero positivo.")
except IndexError:
    print("\nNo se proporcionó VALUE. Usando el valor por defecto: 10^6")
    value = 10**6
except ValueError as e:
    print(f"\nError en el valor de VALUE: {e}. Usando el valor por defecto: 10^6")
    value = 10**6

print(f"\nRunning with {value} elements.")

# Crear los arrays

X = np.arange(value, dtype=np.float64)  # Array en la CPU
Y = cp.arange(value, dtype=cp.float64)  # Array en la GPU



Error en el valor de VALUE: invalid literal for int() with base 10: ''. Usando el valor por defecto: 10^6

Running with 1000000 elements.


### Código Inicial

In [4]:
# Utilizando en secuencial

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("\n Time taken by reduction operation using a function:", tiempo)

print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")

print("-----------------------------------------------------------------------------------------------------------------------------------------------\n")

# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")

print("-----------------------------------------------------------------------------------------------------------------------------------------------\n")

# Utilizando numpy.ndarray.sum()

tiempo= %timeit -r 2 -o -q X.sum()

print("Time taken by reduction operation using numpy.ndarray.sum():", tiempo)

print("Now, the result using numpy.ndarray.sum():", X.sum(), "\n")

print("-----------------------------------------------------------------------------------------------------------------------------------------------\n")


 Time taken by reduction operation using a function: 97 ms ± 1.12 ms per loop (mean ± std. dev. of 2 runs, 10 loops each)
And the result of the sum of numbers in the range [0, value) is: 499999500000.0

-----------------------------------------------------------------------------------------------------------------------------------------------

Time taken by reduction operation using numpy.sum(): 203 µs ± 299 ns per loop (mean ± std. dev. of 2 runs, 1,000 loops each)
Now, the result using numpy.sum(): 499999500000.0 
 
-----------------------------------------------------------------------------------------------------------------------------------------------

Time taken by reduction operation using numpy.ndarray.sum(): 202 µs ± 120 ns per loop (mean ± std. dev. of 2 runs, 1,000 loops each)
Now, the result using numpy.ndarray.sum(): 499999500000.0 

---------------------------------------------------------------------------------------------------------------------------------------

### Código Nuevo 

#### Utilizando Cupy

In [5]:
print("\n Suma utilizando Cupy:")

start_time = time.time()
array_sum = cp.sum(Y, dtype=cp.float64)
time_1 = time.time() - start_time

print(f"Resultado: {array_sum}, Tiempo: {time_1:.6f} segundos\n")

print("---------------------------------------------------------------------\n")


 Suma utilizando Cupy:
Resultado: 499999500000.0, Tiempo: 0.009945 segundos

---------------------------------------------------------------------



#### Utilizando Numba

In [11]:
print("\nSuma utilizando una ufunc personalizada:")

start_time = time.time()
result =  reduc_operation_gpu(X)
time_taken = time.time() - start_time

print(f"Suma total con np.sum y datos transferidos de CUDA: {result}, Tiempo: {time_taken:.6f} segundos")

print("---------------------------------------------------------------------\n")


Suma utilizando una ufunc personalizada:
Suma total con np.sum y datos transferidos de CUDA: 0.0, Tiempo: 0.023904 segundos
---------------------------------------------------------------------



### Análisis Comparativo: Cupy y Numba en la GPU

En esta práctica, se realizaron comparaciones entre los paquetes **Cupy** y **Numba** para realizar una operación de suma en arrays grandes utilizando la GPU. Ambos enfoques fueron evaluados considerando la capacidad de cálculo paralelo de las GPUs y la facilidad de uso que ofrecen estas herramientas.

#### Observaciones sobre Cupy
**Cupy** es una biblioteca diseñada específicamente para GPUs, que replica la funcionalidad de **NumPy** pero optimizada para aprovechar al máximo el hardware de la GPU. Durante esta práctica, la función `cp.sum` se utilizó para calcular la suma directamente en la GPU. Gracias a las optimizaciones internas de **Cupy**, este enfoque fue eficiente y rápido, especialmente para operaciones estándar como la suma de arrays grandes.

#### Observaciones sobre Numba
Por otro lado, **Numba** permite definir funciones personalizadas que se ejecutan en la GPU mediante compilación JIT (*Just-In-Time*). Se implementó una ufunc para realizar la suma del array directamente en la GPU. Aunque **Numba** ofrece mayor flexibilidad y control sobre los cálculos, el rendimiento puede estar condicionado por la implementación de la función y la configuración del hardware. Este enfoque resulta útil para personalizar algoritmos o realizar operaciones más complejas que no están predefinidas.

#### Comparación de Rendimiento
- **Cupy** resultó ser más rápido para operaciones predefinidas como la suma, debido a su diseño optimizado y su integración nativa con GPUs.
- **Numba** se destacó por su capacidad de personalización, lo que es crucial para escenarios donde se requieren operaciones específicas o no estándar.

#### Conclusión
Ambos paquetes son herramientas valiosas para aprovechar el poder computacional de las GPUs:
- **Cupy** es ideal para operaciones estándar y predefinidas, como la suma, gracias a su simplicidad y velocidad.
- **Numba** es más adecuado para algoritmos personalizados o situaciones que requieren un control detallado sobre los cálculos.

La elección entre estos enfoques depende del tipo de tarea y los requisitos del proyecto.
